In [ ]:
# Initial setup: paths and imports
%matplotlib inline
from pathlib import Path
import sys

ROOT = Path.cwd()
SRC_DIR = ROOT / 'src'
if SRC_DIR.exists() and str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))  # allow `import ...` from local src/

IMAGES_DIR = ROOT / 'images1'
print('Ready. Images path:', IMAGES_DIR)

Lists JPGs in IMAGES_DIR and displays the first one (RGB) with title and shape.

In [ ]:
# Preview a sample image (robust if setup cell wasn't run)
from pathlib import Path
from skimage import io
import matplotlib.pyplot as plt

IMAGES_DIR = globals().get('IMAGES_DIR', Path.cwd() / 'images1')

image_files = sorted(IMAGES_DIR.glob('*.jpg'))
assert image_files, f'No .jpg images found in {IMAGES_DIR}'

img_path = image_files[0]
img = io.imread(img_path)  # RGB

plt.imshow(img)
plt.title(img_path.name)
plt.axis('off')
print('Image shape:', img.shape)